In [21]:
from typing import Any, Dict, List, Tuple

import pyomo.environ as pyo
from pyomo.util.infeasible import log_infeasible_constraints


import toml

In [3]:
def get_dsp_usage(PC, PF, cfg: Dict, name: str, board_cfg: Dict):
    """Calculate the DSP usage of a single layer."""
    layer_cfg = cfg["layers"][name]
    layer_type = layer_cfg["TYPE"]
    K = layer_cfg["K"]
    dsp_key = f"b{cfg['global']['BW']}w{cfg['global']['WBW']}"

    if K == 3:
        if layer_type == "STANDARD":
            return board_cfg[dsp_key]["STANDARD_3x3_DSP"] * PC * PF
        if layer_type == "DEPTHWISE_SEPARABLE":
            return (
                board_cfg[dsp_key]["DEPTHWISE_3x3_DSP"] * PC
                + board_cfg[dsp_key]["POINTWISE_DSP"] * PC * PF
            )
        assert False
    assert False


def get_non_negative_divisible(N: int) -> List[int]:
    return [x for x in range(1, N) if N % x == 0]


def get_par_binaries_and_sum(
    model: Any, key: str, layer_name: str, layer_cfg: Dict
) -> Tuple[List[Any], Any]:
    ps = []
    pbs = []
    for i in get_non_negative_divisible(layer_cfg[key[1:]]):
        model.add_component(f"{layer_name}_{key}_{i}", pyo.Var(within=pyo.Binary))
        pbs.append(getattr(model, f"{layer_name}_{key}_{i}"))
        ps.append(pbs[-1] * i)
    p = sum(ps)

    return pbs, p

def get_cycles(PCs, PFs, cfg: Dict, name: str):
    layer_cfg = cfg["layers"][name]
    layer_type = layer_cfg["TYPE"]
    H = layer_cfg["H"]
    W = layer_cfg["W"]
    C = layer_cfg["C"]
    F = layer_cfg["F"]

    PC_divs = get_non_negative_divisible(C)
    PF_divs = get_non_negative_divisible(F)

    if layer_type == "STANDARD" or layer_type == "DEPTHWISE_SEPARABLE":
        return H * W * sum([x * C / y for x, y in zip(PCs, PC_divs)]) * sum([x * C / y for x, y in zip(PFs, PF_divs)]) 
    assert False


In [4]:
cfg = toml.load('../configs/mobilenet-v1.toml')
board_cfg = toml.load('../configs/vu9p.toml')

In [5]:
layer_names = list([x for x in cfg["layers"]])
layer_names

['conv0',
 'conv1',
 'conv2',
 'conv3',
 'conv4',
 'conv5',
 'conv6',
 'conv7',
 'conv8',
 'conv9',
 'conv10',
 'conv11',
 'conv12',
 'conv13']

In [6]:
for i, layer_name in enumerate(cfg['layers']):
    if i > 0:
        if 'INPUT' not in cfg['layers'][layer_name]:
            cfg['layers'][layer_name]['INPUT'] = []
        if not cfg['layers'][layer_name]['INPUT']:
            cfg['layers'][layer_name]['INPUT'].append(layer_names[i-1])

In [55]:
model = pyo.ConcreteModel()

dsp_usages = []
cycles = []
diffs = []
PF_map = {}
for i, layer_name in enumerate(cfg["layers"]):
    layer_cfg = cfg["layers"][layer_name]
    if i == 0:
        PCs, PC = get_par_binaries_and_sum(model, "PC", layer_name, layer_cfg)
        model.add_component(f"{layer_name}_PC", pyo.Constraint(expr=sum(PCs) == 1))
    else:
        PCs, PC = PF_map[layer_cfg['INPUT'][0]]

    PFs, PF = get_par_binaries_and_sum(model, "PF", layer_name, layer_cfg)
    PF_map[layer_name] = PFs, PF

    model.add_component(f"{layer_name}_PF", pyo.Constraint(expr=sum(PFs) == 1))

    dsp_usages.append(
        get_dsp_usage(PC=PC, PF=PF, cfg=cfg, name=layer_name, board_cfg=board_cfg)
    )
    cycles.append(get_cycles(PCs=PCs, PFs=PFs, cfg=cfg, name=layer_name))
    
    # if i % 2 == 1:
    #     model.add_component(f"{layer_name}_abs_cycle_diff", pyo.Var(within=pyo.NonNegativeIntegers))
    #     diffs.append(getattr(model, f"{layer_name}_abs_cycle_diff"))
    #     model.add_component(f"{layer_name}_cycle_diff_leq", pyo.Constraint(expr=- diffs[-1] <= cycles[-1] - cycles[-2]))
    #     model.add_component(f"{layer_name}_cycle_diff_geq", pyo.Constraint(expr=  diffs[-1] >= cycles[-1] - cycles[-2]))

# model.add_component('max_cycle', pyo.Var(within=pyo.NonNegativeIntegers))
# for i, cycle in enumerate(cycles):
#     model.add_component(f"lt_max_{i}", pyo.Constraint(expr=cycle <= model.max_cycle))

model.dsp_usage_cst = pyo.Constraint(expr=sum(dsp_usages) <= board_cfg['DSP'] * 0.8)
model.dsp_usage_cst.display()
# model.OBJ = pyo.Objective(expr=sum(diffs))
model.OBJ = pyo.Objective(expr=sum(cycles))
# model.OBJ = pyo.Objective(expr=model.max_cycle, sense=pyo.minimize)
# model.OBJ = pyo.Objective(expr=model.max_cycle + sum(cycles) / len(cycles), sense=pyo.minimize)
# model.OBJ = pyo.Objective(expr = 0)
# model.OBJ.pprint()
result = pyo.SolverFactory("mindtpy").solve(
    model,
    # strategy='OA',
    time_limit=3600,
    mip_solver='glpk',
    # mip_solver='cplex_persistent',
    # single_tree=True,
    nlp_solver='ipopt',
    tee=True
)
model.OBJ.display()

dsp_usage_cst : Size=1
    Key  : Lower : Body : Upper
    None :  None : None : 5472.0
INFO: ---Starting MindtPy---
INFO: Original model has 16 constraints (1 nonlinear) and 0 disjunctions, with
    116 variables, of which 116 are binary, 0 are integer, and 0 are
    continuous.
INFO: Objective is nonlinear. Moving it to constraint set.
INFO: rNLP is the initial strategy being used.
INFO: Relaxed NLP: Solve relaxed integrality
INFO: Relaxed NLP: OBJ: 1776389.7064099242  LB: -inf  UB: inf  TIME:0.99s
INFO: ---MindtPy main Iteration 1---
INFO: MIP 1: Solve main problem.
INFO: MIP 1: OBJ: 1820447.98835222  LB: 1820447.98835222  UB: inf  TIME: 1.21s
INFO: Fixed-NLP 1: Solve subproblem for fixed integers.
INFO: Fixed-NLP 1: OBJ: 1820447.9999999925  LB: 1820447.98835222  UB:
    1820447.9999999925  TIME: 1.35s
INFO: MindtPy exiting on bound convergence. (UB: 1820447.9999999925 - LB:
    1820447.98835222)/ (1e-10+|bestinteger|:1820447.9999999925) <= relative
    tolerance: 0.001
OBJ : Size=1

In [56]:
pyo.value(model.dsp_usage_cst)

5328.0

In [57]:
for cycle in cycles:
    print(pyo.value(cycle))

14112.0
100352.0
50176.0
200704.0
50176.0
200704.0
50176.0
200704.0
200704.0
200704.0
200704.0
200704.0
50176.0
100352.0


In [58]:
for PFs, PF in PF_map.values():
    print(pyo.value(PF))

8.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
32.0
